In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('cleaned_loan.csv')
df.head()

,loan_amnt,term,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,purpose,total_pymnt,issue_month
0,-0.831337,36,-0.368123,-0.734330,4,-1.611810,2,2,1,-0.615834,12
1,-1.596084,60,0.872219,-1.867178,4,-1.195604,1,0,0,-2.080471,12
2,-1.637050,36,1.057465,-1.511840,4,-2.895639,0,2,11,-1.265084,12
3,0.081095,36,0.394338,0.306146,4,-0.290022,1,2,9,0.267863,12
4,-1.407828,60,0.179561,-1.741542,4,0.577374,1,1,9,-1.124963,12


In [3]:
#create X & y
X = df.drop(columns = ['loan_status'])
y = df['loan_status']

In [4]:
#train-test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = True)

In [10]:
#Hypre-parameter tunning
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

estimator = AdaBoostClassifier(random_state = True)
param_grid = {'n_estimators':list(range(1,30))}
ab_grid = GridSearchCV(estimator,param_grid,cv=5,scoring='accuracy')
ab_grid.fit(X_train,y_train)
ab_grid.best_params_

{'n_estimators': 28}

In [11]:
ab_grid.best_estimator_.feature_importances_

array([0.10714286, 0.03571429, 0.03571429, 0.32142857, 0.        ,
       0.        , 0.        , 0.        , 0.46428571, 0.03571429])

In [12]:
feature = pd.DataFrame(data = ab_grid.best_estimator_.feature_importances_,
                       index = X_train.columns,
                       columns = ['Feature_importances'])

feature_imp = feature[feature['Feature_importances'] > 0 ]
imp_feature = feature_imp.index.tolist()
imp_feature

['loan_amnt', 'term', 'int_rate', 'installment', 'total_pymnt', 'issue_month']

In [14]:
X2 = X[imp_feature]
X2_train,X2_test,y2_train,y2_test = train_test_split(X2,y,test_size = 0.2,random_state = True)

ab_model = AdaBoostClassifier(n_estimators = 28, random_state = True)
ab_model.fit(X2_train,y2_train)

y2_pred_train = ab_model.predict(X2_train)
y2_pred_test = ab_model.predict(X2_test)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
print('train_accuracy:',accuracy_score(y2_train,y2_pred_train))
print('test_accuracy:',accuracy_score(y2_test,y2_pred_test))
print('cv_score:',cross_val_score(ab_model,X2_train,y2_train,cv=5,scoring='accuracy').mean())

train_accuracy: 0.9131337928429799
test_accuracy: 0.9101208459214502
cv_score: 0.9128503873360035
